In [1]:
from pycocotools.coco import COCO
from PIL import Image, ImageDraw
from tqdm import tqdm
import numpy as np
import os
import json
os.chdir('../')

In [3]:
def portion_check(json_file,thrs):
    with open(json_file) as f:
        data = json.load(f)
    count = 0
    for i in data:
        if i['max_obj_area_portion']:
            if i['max_obj_area_portion'] > thrs:
                count += 1
        else:
            pass
    return count/len(data)

def remove_polygon(image, segmentation, background_color=0, seg_or_bbox: str = 'seg'):
    if image.mode == 'L':
        # Convert background color to single-channel grayscale
        background_color = int(background_color[0])
    
    mask = Image.new('L', image.size, 255)
    draw = ImageDraw.Draw(mask)
    
    if seg_or_bbox == 'seg':
        draw.polygon(segmentation, outline=0, fill=0)
    elif seg_or_bbox == 'bbox':
        x, y, w, h = [int(v) for v in segmentation]
        draw.rectangle((x, y, x+w, y+h), outline=0, fill=0)
        
    mask = Image.fromarray(np.array(mask), mode='L')
    
    if image.mode == 'L':
        background = Image.new('L', image.size, background_color)
    else:
        background = Image.new('RGB', image.size, background_color)
    
    removed_image = Image.composite(image, background, mask)
    return removed_image

def crop_polygon(image, segmentation, seg_or_bbox: str = 'seg'):
    background_removed_image = remove_polygon(image, segmentation,(0, 0, 0), seg_or_bbox)
    original_image_array = np.array(image)
    background_removed_image_array = np.array(background_removed_image)
    
    object_only_image_array = original_image_array - background_removed_image_array
    object_only_image = Image.fromarray(object_only_image_array)
    
    return object_only_image

# return the center area of the image
def center_check(midpoint:list, width:int ,height:int ,thrs: float):
    width_area = [0+width*((1-thrs)/2), width-width*((1-thrs)/2)]
    height_area = [0+height*((1-thrs)/2), height-height*((1-thrs)/2)]
    try:
        if midpoint[0] > width_area[0] and midpoint[0] < width_area[1]:
            if midpoint[1] > height_area[0] and midpoint[1] < height_area[1]:
                return True
            else:
                return False
        else:
            return False
    except:
        return False
    
def obj_bg_div(coco_val, json_file_path, og_data_path, save_data_path, seg_or_bbox: str, portion_threshold=0.01, center_threshold=0.7):
    save_data_path = f"{save_data_path}_{portion_threshold}_{seg_or_bbox}_center_{center_threshold}"
    save_data_path_obj = os.path.join(save_data_path, "obj")
    save_data_path_bg = os.path.join(save_data_path, "bg")
    save_data_path_real_bg = os.path.join(save_data_path, "real_bg")
    
    os.makedirs(save_data_path, exist_ok=True)
    os.makedirs(save_data_path_obj, exist_ok=True)
    os.makedirs(save_data_path_bg, exist_ok=True)
    os.makedirs(save_data_path_real_bg, exist_ok=True)

    with open(json_file_path) as json_file:
        json_data = json.load(json_file)

    img_info = []
    num_obj_images , num_bg_images, num_xbg_images, num_real_bg_images = 0, 0, 0, 0
    for entry in tqdm(json_data, desc="Processing images"):
        try:
            max_obj_area_portion = entry['max_obj_area_portion']
            max_obj_midpoint = entry['max_obj_midpoint']
            img_width, img_height = int(entry['width']), int(entry['height'])

            if max_obj_area_portion:
                img_id = entry['img_id']
                img_file = coco_val.loadImgs(img_id)[0]['file_name']
                image_path = os.path.join(og_data_path, img_file)
                segmentation = entry['max_obj_bbox'] if seg_or_bbox == 'bbox' else entry['max_obj_segment_points'][0]

                with Image.open(image_path) as image:
                    if max_obj_area_portion > portion_threshold: # 물체가 이미지의 일정 비율 이상 차지하는 경우
                        cropped_image = crop_polygon(image, segmentation, seg_or_bbox)
                        cropped_image.save(os.path.join(save_data_path_obj, img_file))
                        entry["obj_bg"] = "obj"
                        num_obj_images += 1
                    else:
                        if center_check(max_obj_midpoint, img_width, img_height, center_threshold): # 빈 곳의 중심이 이미지의 중심에 가까운 경우
                            removed_image = remove_polygon(image, segmentation,(0, 0, 0), seg_or_bbox)
                            removed_image.save(os.path.join(save_data_path_bg, img_file))
                            entry["obj_bg"] = "bg"
                            num_bg_images += 1
                        else: # 빈 곳의 중심이 이미지의 중심에 가깝지 않은 경우 (외곽에 위치한 경우)
                            removed_image = remove_polygon(image, segmentation,(0, 0, 0), seg_or_bbox)
                            removed_image.save(os.path.join(save_data_path_bg, img_file))
                            entry["obj_bg"] = "unusable_bg"
                            num_xbg_images += 1
                        
                img_info.append(entry)
            else:  # max obj 비어있는 경우 (그냥 real bg로 저장)
                entry["obj_bool"] = False
                img_file = coco_val.loadImgs(entry['img_id'])[0]['file_name']
                image_path = os.path.join(og_data_path, img_file)

                with Image.open(image_path) as image:
                    image.save(os.path.join(save_data_path_real_bg, img_file))
                img_info.append(entry)
                num_real_bg_images += 1
        except Exception as e:
            print(f"Error processing image {img_file}: {e}")
    

    print(f"Number of object images {num_obj_images}")
    print(f"Number of usable bg images : {num_bg_images}")
    print(f"Number of unusable bg images : {num_xbg_images}")
    print(f"Number of images in 'real_bg' folder: {num_real_bg_images}")
    '''
    print("Saving json file...")
    with open(os.path.join(json_file_path[:-5] + f"_{portion_threshold}_{seg_or_bbox}_center_{center_threshold}.json"), 'w') as outfile:
        json.dump(img_info, outfile)
    print("Done!")
    '''
    return img_info 

In [ ]:
coco_val = COCO("./data/COCO/Annotations/instances_val2014.json")
og_data_path = "./data/COCO/Images/"
save_data_path = "./data/COCO/Images"
json_file_path = "./data/COCO/Annotations/val2014_rmg.json"
seg_or_bbox= "bbox" 
portion_threshold=0.01 
center_threshold=0.7
valid_json = obj_bg_div(coco_val, json_file_path, og_data_path, save_data_path, seg_or_bbox, portion_threshold, center_threshold)

In [ ]:
coco_train = COCO("./data/COCO/Annotations/instances_train2014.json")
og_data_path = "./data/COCO/Images/"
save_data_path = "./data/COCO/Images"
json_file_path = "./data/COCO/Annotations/train2014_rmg.json"
seg_or_bbox= "bbox" 
portion_threshold=0.01 
center_threshold=0.7
train_json = obj_bg_div(coco_train, json_file_path, og_data_path, save_data_path, seg_or_bbox, portion_threshold, center_threshold)

In [7]:
train_json.extend(valid_json)
final_json = {} 
for i in train_json:
    final_json[i['img_id']] = i
    
json_file_path = "./data/COCO/Annotations/train2014_rmg.json"
print("Saving json file...")
with open(os.path.join(json_file_path[:-5] + f"_{portion_threshold}_{seg_or_bbox}_center_{center_threshold}.json"), 'w') as outfile:
    json.dump(final_json, outfile)
print("Done!")